# Initalize libraries

## Import libraries

In [ ]:
import sys, os
import gc
import time
from os.path import join
from os import path
from importlib import reload
from getpass import getuser
from glob import glob
from tqdm.auto import tqdm
from joblib import Parallel, delayed

# Data
import xarray as xr
import h5py
import numpy as np
from nexusformat.nexus import *
#from sfdata import SFDataFiles

# Plotting
import imageio
import matplotlib.pyplot as plt
import fabio
import skimage.morphology

# skimage
from skimage.draw import ellipse

# scipy
from scipy.ndimage.filters import gaussian_filter
import scipy
from scipy import stats

# pyFAI
import pyFAI
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.detectors import Detector

# Self-written libraries
sys.path.append(join(os.getcwd(), "library"))
import fthcore as fth
import helper_functions as helper
import mask_lib
import interactive
from interactive import cimshow
import reconstruct_rb as rec
import reconstruct as reco

# Facility specific loading functions
import MAXI_loading as loading

plt.rcParams["figure.constrained_layout.use"] = True  # replaces plt.tight_layout

In [ ]:
# Is there a GPU?
try:
    # Cupy
    import cupy as cp
    import cupyx as cpx

    GPU = True

    print("GPU available")

    # Self-written library
    import CCI_core_cupy as cci
    import Phase_Retrieval as PhR
except:
    GPU = False
    import CCI_core as cci
    import Phase_Retrieval_noGPU as PhR

    print("GPU unavailable")

In [ ]:
# interactive plotting
import ipywidgets

%matplotlib widget

# Auto formatting of cells
#%load_ext jupyter_black

## Experiment specific Functions

In [ ]:
PROPOSAL = 11018955
USER = getuser()

BASEFOLDER = "/asap3/petra3/gpfs/p04/2024/data/%s/" % PROPOSAL
sample_name = "2403_tomo_"
spefile_name = "2403_tomo"
single_frame_folder = "cmos"

# Load mnemonics for easy data handling
mnemonics = loading.mnemonics

### Loading data

In [ ]:
def load_data(scan_id, keypath = mnemonics["pre_scan_snapshot"], keys = None):
    """
    Load data of all specified keys from keypath
    
    Parameter
    =========
    scan_id : int
        experimental identifier of scan
    keypath : str
        path of nexus file tree to relevant data field
    keys : str or list of strings
        keys to load from keypath
        
    Output
    ======
    data : dict
        data dictionary of keys
    ======
    author: ck 2024
    """
        
    #Generate filename from scan_id
    fname = loading.generate_filename(join(BASEFOLDER,"raw"), sample_name, ".h5", scan_id)
    
    # load data with basic loading function
    data = loading.load_data(fname, keypath, keys = keys)
    
    return data


def load_key(scan_id, key):
    """
    Load any kind of data specified by key (path)
    
    Parameter
    =========
    scan_id : int
        experimental identifier of scan
    key : str
        key path of nexus file tree to relevant data field
   
    Output
    ======
    data : dict
        data dictionaray on single key
    ======
    author: ck 2024
    """
    #Generate filename from scan_id
    fname = loading.generate_filename(join(BASEFOLDER,"raw"), sample_name, ".h5", scan_id)
    
    # load data with basic loading function
    data = loading.load_key(fname, key)
    
    return data

def get_spe_id(im_id,camera_type = "ccd"):
    """
    Returns spe_id or list of spe_ids for given image id
    
    Parameter
    =========
    im_id : int
        experimental identifier of image scan
    camera_type : str
        "ccd" or "cmos", select corresponding kezy from mnemonics dict
   
    Output
    ======
    spe_ids_list : list of str
        contains all spe ids the correspond to the scan
    ======
    author: ck 2024
    """
    #Generate filename from scan_id
    fname = loading.generate_filename(join(BASEFOLDER,"raw"), sample_name, ".h5", im_id)
    
    # Load spe filenames
    spe_ids = loading.load_key(fname,mnemonics[camera_type])
    
    # If it contains multiple images loop over all
    if not np.array(spe_ids).shape:
        spe_ids_list = [str(spe_ids)[-9:-5]]
    else:
        # Combine all spe id in list
        spe_ids_list = []
        for spe_id in spe_ids:
            spe_ids_list.append(str(spe_id)[-9:-5])
    return spe_ids_list


def load_images(im_id, camera_type = "ccd"):
    """
    Loads all images of a given image id
    
    Parameter
    =========
    im_id : int
        experimental identifier of image scan
    camera_type : str
        "ccd" or "cmos", select corresponding kezy from mnemonics dict
   
    Output
    ======
    images : array
        stack of all images the correspond to the id
    ======
    author: ck 2024
    """
    
    # Find spe files
    spe_ids_list = get_spe_id(im_id,camera_type = camera_type)
    
    # Loop over image spe files
    images = []
    
    # Load ccd or cmos data
    if camera_type == "ccd":
        for spe_id in spe_ids_list:
            fname = join(BASEFOLDER, "raw", spefile_name + "  " + spe_id + ".spe")
            images.append(np.squeeze(np.array(imageio.mimread(fname, memtest="5000MB"))))

    return np.stack(images)

In [ ]:
# Full image loading procedure
def load_processing(im_id, crop=None):
    """
    Loads images, averaging of two individual images (scans in tango consist of two images),
    padding to square shape, Additional cropping (optional)
    """

    # Load data
    images = load_images(im_id)

    # Zeropad to get square shape
    #images = helper.padding(images)

    # Calculate mean
    if images.ndim > 2:
        image = np.mean(images, axis=0)
    else:
        image = images.copy()

    # Optional cropping
    if crop is not None:
        images = images[:, :crop, :crop]
        image = image[:crop, :crop]

    return image, images

### Loading, saving fth & cdi data

In [ ]:
# Saving of log files for fth and cdi recos
def save_fth_h5():
    # Save h5
    data = {}
    data["im_id"] = im_id
    data["topo_id"] = topo_id
    data["topo_centered"] = topo_c
    data["im_centered"] = im_c
    data["holo"] = holo
    data["recon"] = recon
    data["factor"] = factor
    data["offset"] = offset
    data["center"] = center
    data["roi"] = roi
    data["prop_dist"] = prop_dist
    data["phase"] = phase
    data["mask_bs"] = mask_pixel_smooth
    data["bs_smoothing"] = bs_smoothing
    data["experimental_setup"] = experimental_setup

    filename = join(
        folder_general, "Logs", "Data_ImId_%04d_RefId_%s_%s" % (im_id, topo_id, USER)
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)


def save_cdi_h5():
    # Save h5
    data = {}
    data["im_id"] = im_id
    data["topo_id"] = topo_id
    data["pos"] = pos
    data["neg"] = neg
    data["factor"] = factor
    data["offset"] = offset
    data["center"] = center
    data["roi"] = roi
    data["prop_dist"] = prop_dist_cdi
    data["phase"] = phase_cdi
    data["mask_bs"] = mask_bs_cdi
    data["supportmask"] = supportmask
    data["mask_pixel"] = mask_pixel
    data["p_pc"] = p_pc
    data["n_pc"] = n_pc
    data["experimental_setup"] = experimental_setup

    filename = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_RefId_%s_cdi_%s" % (im_id, topo_id, USER),
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)
    return


def save_topo_holo(topo_c, pos_id, neg_id):
    """
    Save only topo holos which can be later used for single helicity reconstructions
    """
    data = {}
    data["pos_id"] = pos_id
    data["neg_id"] = neg_id
    data["topo"] = topo_c

    filename = join(
        folder_general,
        "Topos",
        "Topo_ImId_%04d_RefId_%04d_cdi_%s" % (pos_id, neg_id, USER),
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)
    return

def load_fth(im_id, topo_id):
    """
    Load fth dataset
    """
    fname = join(
        folder_target,
        "Logs",
        "Data_ImId_%s_RefId_%s_%s.hdf5"
        % (
            im_id,
            topo_id,
            USER,
        ),
    )

    with h5py.File(fname, "r") as f:
        data = {}
        for key in f.keys():
            if key != "experimental_setup":
                data[key] = f[key][()]

    return data

def load_topo_holo(pos_id, neg_id):
    """
    Load topo holos for single helicity reconstructions
    """
    fname = join(
        folder_general,
        "Topos",
        "Topo_ImId_%04d_RefId_%04d_cdi_%s.hdf5" % (pos_id, neg_id, USER),
    )

    with h5py.File(fname, "r") as f:
        im_out = f["topo"][()]
    return im_out


def load_cdi(im_id, topo_id):
    """
    Load cdi dataset
    """
    fname = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_RefId_%s_cdi_%s.hdf5" % (im_id, topo_id, USER),
    )

    with h5py.File(fname, "r") as f:
        data = {}
        for key in f.keys():
            if key != "experimental_setup":
                data[key] = f[key][()]
    return data_cdi

### Other

In [ ]:
# Worker which performs complete fth reconstruction process
def worker(image, topo):
    # Centering
    shift_c = np.array(topo.shape) / 2 - center
    topo_c = cci.shift_image(topo, shift_c)
    im_c = cci.shift_image(image, shift_c)
    
    ## Image registration
    shift = cci.image_registration(
        im_c[roi_im_reg],
        topo_c[roi_im_reg],
        method="dipy",
    )

    print("Relative shift is: %s"%shift)

    
    # Correct relative drift
    topo_c = cci.shift_image(topo_c, shift) 
    
    # Optional ewalds sphere projection
    if project_ewalds_sphere is True:
        %%time
        proj_im = PhR.inv_gnomonic2(im_c,alpha=experimental_setup["angle"], center=None, experimental_setup = experimental_setup, method='cubic' , mask=None)
        proj_topo = PhR.inv_gnomonic2(topo_c,alpha=experimental_setup["angle"], center=None, experimental_setup = experimental_setup, method='cubic' , mask=None)

        im_c = proj_im.copy()
        topo_c = proj_topo.copy()

    # Create masks
    # Load polygon mask and shift to manually determined position
    mask = mask_lib.load_poly_masks(
    experimental_setup["binning"] * np.array(image.shape),
    load_poly_coordinates(),
    polygon_names,)
    mask = np.round(cci.shift_image(mask,mask_shift))
    
    # Increase/Decrease mask size
    footprint = skimage.morphology.disk(np.abs(mask_scale))
    if mask_scale > 0:
        mask = skimage.morphology.dilation(mask, footprint) # increase size
    elif mask_scale < 0: 
        mask = skimage.morphology.erosion(mask, footprint) # decrease size

    # Create image specific beamstop mask
    mask_im = mask.copy()
    mask_im = mask_im + (im_c > experimental_setup["oversaturation"])
    mask_im[mask_im > 1] = 1

    # Create topo specific beamstop mask
    mask_topo = mask.copy()
    mask_topo = mask_topo + (topo_c > experimental_setup["oversaturation"])

    # Combine both
    mask_pixel = mask_im + mask_topo
    mask_pixel[mask_pixel > 1] = 1
    
    # Create smooth mask
    footprint = skimage.morphology.disk(4)
    mask_pixel_smooth = skimage.morphology.dilation(mask_pixel, footprint)
    mask_pixel_smooth = gaussian_filter(mask_pixel_smooth, 2)

    # Get scaling factor and offset
    factor, offset = cci.dyn_factor(
        im_c * (1 - mask_pixel),
        topo_c * (1 - mask_pixel),
        method="correlation",
        verbose=False,
        plot=False,
    )

    # Calculate differences (magnetic) and sums (topographc) contrast holograms.
    # _c: centered, without beamstop, _b: centered, with beamstop
    diff_c = im_c / factor - topo_c - offset
    sum_c = im_c / factor + topo_c - offset

    if heraldo is True:
        #holo = sum_c * heraldo_operator * mask_bs*(1 - mask_pixel_smooth)
        holo = diff_c * heraldo_operator * (1 - mask_pixel_smooth)
    elif heraldo is False:
        #holo = sum_c * mask_bs*(1 - mask_pixel_smooth)
        holo = diff_c * (1 - mask_pixel_smooth)

    # Reconstruct
    recon = cci.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )

    # worker dictionary
    worker_dict = {}
    worker_dict["center"] = center
    worker_dict["topo_c"] = topo_c
    worker_dict["im_c"] = im_c
    worker_dict["recon"] = recon
    worker_dict["factor"] = factor
    worker_dict["offset"] = offset
    worker_dict["shift"] = shift
    worker_dict["diff_c"] = diff_c
    worker_dict["sum_c"] = sum_c
    worker_dict["mask_bs"] = mask_pixel_smooth
    worker_dict["mask_pixel"] = mask_pixel
    
    return worker_dict

In [ ]:
# Setup phase and propagation for cdi once
phase_cdi = 0
prop_dist_cdi = 0
dx = 0
dy = 0


def phase_retrieval(
    pos, neg, mask_pixel, supportmask, vmin = 0, Startimage=None, Startgamma=None
):
    # Prepare Input holograms
    pos2 = pos.copy()
    neg2 = neg.copy()

    mi, _ = np.percentile(pos2[pos2 != 0], [vmin, 99.9])
    pos2 = pos2 - mi
    mi, _ = np.percentile(neg2[neg2 != 0], [vmin, 99.9])
    neg2 = neg2 - mi

    pos2[pos2 < 0] = 0
    neg2[neg2 < 0] = 0
    pos2 = pos2.astype(complex)
    neg2 = neg2.astype(complex)

    bsmask_p = mask_pixel.copy()
    bsmask_p[pos2 <= 0] = 1
    bsmask_n = mask_pixel.copy()
    bsmask_n[neg2 <= 0] = 1

    # Setup start image and startgamma
    if Startimage is None:
        Startimage = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(supportmask)))
    else:
        Startimage = Startimage.copy()
    if Startgamma is None:
        Startgamma = np.ones(pos.shape) * 1e-6 * 2
        Startgamma[pos.shape[0] // 2, pos.shape[1] // 2] = 0.7
    else:
        Startgamma = Startgamma.copy()

    # Settings for phase retrieval reconstructions
    partial_coherence = True

    # Setup
    retrieved_p = np.zeros(pos2.shape, np.cdouble)
    retrieved_n = np.zeros(pos2.shape, np.cdouble)

    # Algorithms and Inital guess
    plt.rcParams["figure.dpi"] = 100
    print("CDI - larger mask")

    algorithm_list = ["mine", "mine", "mine"]
    Nit_list = [700, 50, 50]  # iterations for algorithm_list

    x = (np.sqrt(np.maximum(pos2, np.zeros(pos2.shape)))[mask_pixel == 0]).flatten()
    y = ((np.abs(Startimage))[mask_pixel == 0]).flatten()
    res = stats.linregress(x, y)
    Startimage -= res.intercept
    Startimage /= res.slope

    average_img = 30
    real_object = False  # always set to False

    if partial_coherence:
        RL_freq = 20
        RL_it = 50

        algorithm_list_pc = ["mine", "ER", "ER"]
        Nit_list_pc = [700, 50, 50]

    # Execute Phase retrieval
    start_time = time.time()
    for i in range(len(Nit_list) // 3):
        print("############ -   CDI")

        # Positive helicity - beta_mode="arctan"
        retrieved_p, Error_diff_p, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(pos2, np.zeros(pos2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i],
            beta_zero=0.5,
            Nit=Nit_list[3 * i],
            beta_mode="arctan",
            plot_every=349,
            Phase=Startimage,
            seed=False,
            real_object=real_object,
            bsmask=bsmask_p,
            average_img=average_img,
            Fourier_last=True,
        )

        # Positive helicity - beta_mode="const"
        retrieved_p, Error_diff_p2, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(pos2, np.zeros(pos2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i + 1],
            beta_zero=0.5,
            Nit=Nit_list[3 * i + 1],
            beta_mode="const",
            plot_every=24,
            Phase=retrieved_p,
            seed=False,
            real_object=real_object,
            bsmask=bsmask_p,
            average_img=average_img,
            Fourier_last=True,
        )

        # Negative helicity - beta_mode="arctan"
        retrieved_n, Error_diff_n2, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(neg2, np.zeros(neg2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i + 2],
            beta_zero=0.5,
            Nit=Nit_list[3 * i + 2],
            beta_mode="const",
            plot_every=24,
            Phase=retrieved_p * np.sqrt(np.sum(neg2) / np.sum(pos2)),
            seed=False,
            real_object=real_object,
            bsmask=bsmask_n,
            average_img=average_img,
            Fourier_last=True,
        )

        print("--- %s seconds ---" % np.round((time.time() - start_time), 2))

        Startimage = retrieved_p.copy()

        # Partial coherence phase retrieval
        if partial_coherence:
            # CDI_PC
            print("############   -   CDI_pc")
            pos3 = (np.abs(retrieved_p) ** 2) * bsmask_p + np.maximum(
                pos2, np.zeros(pos2.shape)
            ) * (1 - bsmask_p)
            neg3 = (np.abs(retrieved_n) ** 2) * bsmask_n + np.maximum(
                neg2, np.zeros(neg2.shape)
            ) * (1 - bsmask_n)

            # retrieve pos image
            (
                retrieved_p_pc,
                Error_diff_p_pc,
                Error_supp,
                gamma_p,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(pos3),
                mask=supportmask,
                mode=algorithm_list_pc[3 * i],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i],
                beta_mode="arctan",
                gamma=Startgamma,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=349,
                Phase=Startimage,
                seed=False,
                real_object=False,
                bsmask=np.zeros(bsmask_p.shape),
                average_img=average_img,
                Fourier_last=True,
            )

            (
                retrieved_p_pc,
                Error_diff_p_pc2,
                Error_supp,
                gamma_p,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(pos3),
                mask=supportmask,
                mode=algorithm_list[3 * i + 1],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i + 1],
                beta_mode="const",
                gamma=gamma_p,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=24,
                Phase=retrieved_p_pc,
                real_object=False,
                bsmask=np.zeros(bsmask_p.shape),
                average_img=average_img,
                Fourier_last=True,
            )
            (
                retrieved_n_pc,
                Error_diff_n_pc2,
                Error_supp,
                gamma_n,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(neg3),
                mask=supportmask,
                mode=algorithm_list[3 * i + 2],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i + 2],
                beta_mode="const",
                gamma=gamma_p,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=24,
                Phase=retrieved_p_pc * np.sqrt(np.sum(neg2) / np.sum(pos2)),
                real_object=False,
                bsmask=np.zeros(bsmask_n.shape),
                average_img=average_img,
                Fourier_last=True,
            )

            print("--- %s seconds ---" % np.round((time.time() - start_time), 2))

            Startimage = retrieved_p_pc.copy()
            Startgamma = gamma_p.copy()

    print("Phase Retrieval Done!")

    return (
        retrieved_p,
        retrieved_n,
        retrieved_p_pc,
        retrieved_n_pc,
        bsmask_p,
        bsmask_n,
        gamma_p,
        gamma_n,
    )

# Experimental Details

In [ ]:
# Dict with most basic experimental parameter
experimental_setup = {
    "ccd_dist": 0.20,  # ccd to sample distance
    "px_size": 13.5e-6,  # pixel_size of camera
    "binning": 1,  # Camera binning
    "oversaturation" : 60e3, # Pixel saturation threshold
    "energy": 778
}

# Setup for azimuthal integrator
detector = Detector(
    experimental_setup["binning"] * experimental_setup["px_size"],
    experimental_setup["binning"] * experimental_setup["px_size"],
)
experimental_setup["z_angle_offset"] = 67

# General saving folder and log folder
folder_general = helper.create_folder(join(BASEFOLDER, "processed"))
helper.create_folder(join(folder_general, "Logs"))
helper.create_folder(join(folder_general, "Topos"))
print("Output Folder: %s" % folder_general)

# Load images

Start by loading the images: image of interest (im), reference of charge scattering (topo), any kind of dark image (dark)

We estalished the following convention: Difference Hologram which contains only the magnetic scattering will be calculated according to:

$Diff = \frac{Image}{factor} - Topo$,

where the factor is used for intensity scaling. In Case that you recorded scans of the same magnetic state with both helicities, use the image with negative helicity as topo and the one with positive helicity as image

In [ ]:
# Define ids of different run numbers for image loading
im_id = [
    749,
    744,
]  # Convention for stitching, entry 0: Largest Beamstop, 1: Smaller beamstop, n: smallest beamstop

topo_id = [
    747,
    746,
]  # single helicity mode: image without magnetic contrast, double helicity: neg

dark_id = [748,745]  # camera background

# Other loading
# Load energy and add to experimental setup
experimental_setup["energy"] = load_key(im_id[0],mnemonics["energy"])
experimental_setup["lambda"] = helper.photon_energy_wavelength(
    experimental_setup["energy"], input_unit="eV"
)

## Load dark image

In [ ]:
# Load image
if dark_id is not None:
    dark = []
    for idx in dark_id:
        arr, _ =load_processing(idx, crop=None)

        dark.append(arr)
        
    dark = np.stack(dark)
    # Plot
    fig, ax = cimshow(dark)
    ax.set_title("Dark")

## Load image data

In [ ]:
# Load all images
images = []
for i, idx in enumerate(tqdm(im_id)):
    image, _ = load_processing(idx, crop=None)
    image = image - dark[i]
    images.append(image)
images = np.stack(images)
    
# Plot
fig, ax = cimshow(images)
ax.set_title("Image")

## Load topo data set and average

In [ ]:
# Load all images
topos = []
for i,idx in enumerate(tqdm(topo_id)):
    topo, _ = load_processing(idx, crop=None)
    topos.append(topo - dark[i])
topos = np.stack(topos)
    
# Plot
fig, ax = cimshow(topos)
ax.set_title("Topo")

In [ ]:
# Thresholding to prevent negative values
topos = np.array(topos)
images = np.array(images)
topos[topos < 1] = 0
images[images < 1] = 0

fig, ax = cimshow(images)
ax.set_title("Image with subtracted dark")

## Setup xarray

In [ ]:
# Setup xarray for images of different beamstops
data = xr.Dataset()

# Coordinates
data["scan_id"] = im_id

# Fill with data
data["images"] = xr.DataArray(images, dims=["scan_id", "y", "x"])
data["topos"] = xr.DataArray(topos, dims=["scan_id", "y", "x"])

# Get convenient data variables as np arrays
image = data["images"][0].values.copy()
topo = data["topos"][0].values.copy()

# Add attributes
data = data.assign_attrs({"im_ids": im_id, "topo_ids": topo_id, "dark_ids": dark_id})
data = data.assign_attrs(experimental_setup)

In [ ]:
# Show comparision between data for stitching and normal holograms
idx_1 = 0  # Image for stitching
idx_2 = 1  # Any other image

fig, ax = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
mi, ma = np.percentile(data["images"][idx_1], [0.1, 99.99])
ax[0, 0].imshow(data["images"][idx_1], vmin=mi, vmax=ma)
ax[0, 0].set_title("Image")
mi, ma = np.percentile(data["images"][idx_2], [0.01, 99.99])
ax[0, 1].imshow(data["images"][idx_2], vmin=mi, vmax=ma)
ax[0, 1].set_title("Image for stitching")
mi, ma = np.percentile(data["topos"][idx_1], [0.1, 99.99])
ax[1, 0].imshow(data["topos"][idx_1], vmin=mi, vmax=ma)
ax[1, 0].set_title("Topo")
mi, ma = np.percentile(data["topos"][idx_2], [0.01, 99.99])
ax[1, 1].imshow(data["topos"][idx_2], vmin=mi, vmax=ma)
ax[1, 1].set_title("Topo for stitching")

# Center holograms

* Find center of the hologram to get a well-defined q-space. 
* Create smooth mask for beamstop or overexposed areas in direct beam

## Basic widget to find center

Try to **align** the circles to the **center of the scattering pattern**. Care! Position of beamstop might be misleading and not represent the actual center of the hologram. Circles are just a guide to eye and will not be used otherwise.

In [ ]:
# Find center position via widget
c0, c1 = [1009, 1101]  # initial values
ic = interactive.InteractiveCenter(images[-1], c0=c0, c1=c1)

In [ ]:
# Get center positions
center = [ic.c0, ic.c1]
print(f"Center:", center)

## Azimuthal integrator widget for finetuning
More of an "expert widget" which works very well for alignment if you have an Airy Pattern as a scattering image. PyFai transforms images from carthesian detector coordinate system into polar coordinate system with angle `phi` and radial distance `q` as axis (azimuthal transformation). The center of the coordinate system will be defined in the azimuthal integrator class and must not necessarily represents the center coordinates of your image array. If the center is set correctly, all rings of the Airy pattern will be transformed into a straight line in the I(q,chi)-plot as rings appear at a given q for all angles chi.

In [ ]:
# Setup azimuthal integrator for virtual geometry
ai = AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=experimental_setup["lambda"],
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Not the widget, just for double checking to find correct radial range for plotting
# Perform azimuthal transformation
I_t, q_t, phi_t = ai.integrate2d(
    np.log10(image - np.min(image) + 1),
    500,  # number of points for phi
    radial_range=(0.01, 0.3),  # relevant q-range
    unit="q_nm^-1",
    correctSolidAngle=False,
    method="BBox",
)
# Combine in an xarray for plotting
az2d = xr.DataArray(I_t, dims=("phi", "q"), coords={"q": q_t, "phi": phi_t})

# Plot
fig, ax = plt.subplots()
mi, ma = np.percentile(I_t, [1, 95])
az2d.plot.imshow(ax=ax, vmin=mi, vmax=ma)
plt.title(f"Azimuthal integration")

In [ ]:
# The widget
aic = interactive.AzimuthalIntegrationCenter(
    np.log10(images[-1] - np.min(images[-1]) + 1),
    # image,
    ai,
    c0=center[0],
    c1=center[1],
    im_data_range=[1, 98],
    radial_range=(0.0, 0.05),
    qlines=[100, 110],
)

In [ ]:
# Get center positions from widget
center = [aic.c0, aic.c1]
print(f"Center:", center)
data = data.assign_attrs({"center": center})

## Centering of image hologram

In [ ]:
# Center all images
# Find shift vector
shift_c = np.array(image.shape) / 2 - center

# Apply centering
for key in ["images", "topos"]:
    tmp = data[key].values.copy()
    tmp = cci.shift_image_stack(tmp, np.tile(shift_c, (tmp.shape[0], 1)))

    data[key] = data[key].copy(data=tmp)

    # Plot centered images and topo
    fig, ax = cimshow(data[key])
    ax.set_title(key)

# Get convenient variables
im_c = data["images"].copy()
topo_c = data["topos"].copy()

# Create beamstops

We want to cover the beamstop with a smooth circle to cover its sharp edges as these would create ringing-like artifacts in the reconstruction plane. Make it only as large as necessary to keep as much information as possible.

## Manual masking

In [ ]:
# Draw masks for each beamstop
poly_masks = []
for i in range(len(data["scan_id"])):
    print("Scan_ids: %d - %d"%(data.attrs["im_ids"][i],data.attrs["topo_ids"][i]))
    poly_masks.append(interactive.draw_polygon_mask(im_c[i]))

In [ ]:
# Take poly coordinates and mask from widget
p_coord, mask_draw = [],[]
for i in range(len(data["scan_id"])):
    p_coord.append(poly_masks[i].get_vertice_coordinates())
    mask_draw.append(poly_masks[i].full_mask.astype(int))

print("Copy these coordinates into the 'load_poly_coordinates()' function:")
[print("Beamstop %d: %s\n"%(i,p_coord[i])) for i in range(len(p_coord))]


# Plot image with beamstop and valid pixel mask
for image_number in range(len(data["scan_id"])):
    fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
    fig.suptitle("Scan_ids: %d - %d"%(data.attrs["im_ids"][image_number],data.attrs["topo_ids"][image_number]))
    mi, ma = np.percentile(im_c[image_number] * (1 - mask_draw[image_number]), [0.1, 99.9])
    ax[0].imshow(im_c[image_number] * (1 - mask_draw[image_number]), cmap="viridis", vmin=mi, vmax=ma)
    ax[0].set_title("Image * (1-mask_draw)")

    mi, ma = np.percentile(im_c[image_number] * mask_draw[image_number], [0.1, 99.9])
    ax[1].imshow(im_c[image_number] * mask_draw[image_number], vmin=mi, vmax=ma)
    ax[1].set_title("Image * mask_draw")

    ax[2].imshow(1 - mask_draw[image_number])
    ax[2].set_title("1 - mask_draw")

In [ ]:
def load_poly_coordinates():
    """
    Dictionary that stores polygon corner coordinates of all drawn masks
    """

    # Setup dictonary
    mask_coordinates = dict()
    mask_coordinates["large_bs"] = [[(864.2, 894.5), (854.7, 907.0), (840.0, 929.0), (827.6, 947.3), (818.0, 971.4), (817.3, 983.2), (803.4, 989.8), (773.4, 989.8), (521.4, 986.8), (471.6, 985.4), (243.6, 980.0), (114.8, 974.3), (41.9, 972.8), (-4.5, 970.5), (-5.2, 1034.9), (155.0, 1042.1), (389.4, 1048.8), (606.7, 1052.0), (765.7, 1052.3), (798.6, 1052.3), (817.2, 1061.6), (832.9, 1102.4), (845.8, 1126.7), (883.7, 1168.2), (924.5, 1191.0), (981.7, 1206.8), (1023.6, 1204.9), (1073.7, 1189.9), (1103.7, 1171.3), (1135.9, 1139.1), (1153.8, 1109.7), (1171.7, 1066.1), (1181.7, 1056.1), (1285.4, 1059.7), (1385.9, 1057.8), (1617.6, 1055.7), (1833.9, 1052.1), (1994.8, 1048.5), (2056.9, 1048.6), (2055.4, 983.5), (2048.8, 985.6), (1732.7, 992.8), (1618.2, 995.6), (1548.7, 995.5), (1401.3, 996.2), (1268.3, 996.2), (1186.2, 996.9), (1173.8, 990.3), (1167.2, 980.1), (1149.7, 935.6), (1116.9, 891.2), (1065.9, 858.3), (1047.7, 849.6), (1033.8, 847.4), (1027.3, 837.9), (1028.7, 696.5), (1029.9, 411.7), (1031.8, 163.2), (1031.7, -7.7), (971.8, -5.5), (970.4, 145.6), (966.7, 517.2), (964.6, 782.6), (963.9, 831.4), (951.5, 842.7), (895.0, 869.9)]]
    mask_coordinates["small_bs"] = [[(1017.3, 1002.6), (1010.3, 1006.1), (1003.5, 1013.2), (1002.3, 1019.5), (1001.2, 1026.9), (1002.3, 1033.4), (1010.5, 1045.0), (1017.0, 1049.3), (1024.0, 1051.0), (1035.2, 1048.2), (1044.9, 1039.2), (1047.1, 1031.2), (1047.7, 1019.7), (1044.5, 1011.9), (1038.0, 1007.1), (1031.3, 1002.5), (1022.7, 1001.8)], [(1028.1, 992.7), (1024.2, 996.7), (1024.9, 1001.1), (1025.3, 1003.8), (1030.9, 1003.5), (1032.3, 999.1), (1031.4, 993.7)], [(1028.2, 976.3), (1025.5, 978.3), (1026.2, 982.3), (1028.7, 983.7), (1031.7, 983.1), (1033.0, 979.1), (1031.6, 976.6)], [(1029.6, 964.7), (1025.6, 968.8), (1026.4, 972.7), (1029.4, 973.7), (1032.5, 971.9), (1032.1, 967.3)]]
    return mask_coordinates

# Check Poly Mask 

In [ ]:
polygon_names = [
    ["large_bs"],  # list of masks for first image,
    ["small_bs"]   # list of masks for second image
]
mask_draw = []  # now a list of masks for all the
for image_number, poly_masks in enumerate(polygon_names):
    mask_draw.append(
        mask_lib.load_poly_masks(
            im_c[image_number].shape,
            load_poly_coordinates(),
            poly_masks,
        )
    )
mask_draw = np.stack(mask_draw)

# Assign to xarray
data["mask_pixel"] = xr.DataArray(mask_draw,dims=["scan_id","y","x"])
mask_pixel = data["mask_pixel"].sum("scan_id").values
mask_pixel[mask_pixel>1] = 1

for image_number in range(len(polygon_names)):
    # Plot image with beamstop and valid pixel mask
    fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
    mi, ma = np.percentile(
        im_c[image_number] * (1 - mask_draw[image_number]), [0.1, 99.9]
    )
    ax[0].imshow(
        im_c[image_number] * (1 - mask_draw[image_number]),
        cmap="viridis",
        vmin=mi,
        vmax=ma,
    )
    ax[0].set_title("Image %s * (1-mask_draw)" % image_number)

    mi, ma = np.percentile(im_c[image_number] * mask_draw[image_number], [0.1, 99.9])
    ax[1].imshow(im_c[image_number] * mask_draw[image_number], vmin=mi, vmax=ma)
    ax[1].set_title("Image %s * mask_draw" % image_number)

    ax[2].imshow(1 - mask_draw[image_number])
    ax[2].set_title("1 - mask_draw")

# Image Registration

Relative drift between data holograms and their corresponding topo holograms is calculated by image registration algorithm. Necessary to get well defined difference hologram. The reference is always the static background image (topo).

## Set Alignment ROI 

Set a region of interest (ROI) of reference (topo) use for image registration is performed.

How to use:
1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
# We need a roi to calc alignment between image and topo for each beamstop
rois = []
fig, ax = interactive.cimshow(images)  # , {"cmap": "gray"})

In [ ]:
# Takes start and end of x and y axis
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi_im_reg = np.array([y1, y2, x1, x2]).astype(int)
rois.append(roi_im_reg)

print(f"Image registration roi:", rois)

In [ ]:
# Convert rois to np slice object
data["roi_alignment"] = xr.DataArray(np.stack(rois),dims=["stitch_pair","roi_px"])
for i, roi in enumerate(rois):
    rois[i] = np.s_[roi[0] :roi[1], roi[2] : roi[3]] 

## Calculate drift of images

In [ ]:
# Calc shifts
for key in ["images","topos"]:
    shifts = []
    for i in range(len(data.scan_id)-1):
        print(f"Image registration roi:", rois[i])
        shifts.append(
            cci.image_registration(
                data[key][i][rois[i]].values,
                data[key][i+1][rois[i]].values,
                method="dipy",
            )
        )
        print("Relative shift is: %s" % shifts)
    
    # assign to xarray
    data["shift_"+key] = xr.DataArray(shifts,dims=["stitch_pair","shift_px"])

In [ ]:
for image_number in range(len(data.scan_id)-1):
    for key in ["images","topos"]:
        shift = data["shift_"+key][image_number].values
        
        # Define shift manually for comparison
        tmp_shift = [0.03, 20]

        # Loop over shifts
        temp_diff = np.zeros((3, data[key][image_number].shape[0], data[key][image_number].shape[1]))
        shift_tmp = [
            [0, 0],
            tmp_shift,
            -shift,
        ]
        
        for i, tshift in enumerate(shift_tmp):
            temp = cci.shift_image(data[key][image_number].values, tshift)
            temp_factor = cci.dyn_factor(temp[rois[image_number]], data[key][image_number+1].values[rois[image_number]],verbose=True)
            temp_diff[i] = temp - temp_factor[0] * data[key][image_number+1].values

        # Plots for comparision
        fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12, 4))
        fig.suptitle("Scan_ids: %d - %s"%(data.attrs["im_ids"][image_number],key))
        mi, ma = np.percentile(temp_diff[0]*(1-mask_pixel), [0.1, 99.9])
        ax[0].imshow(temp_diff[0], vmin=mi, vmax=ma)
        ax[0].set_title("Zero shift")
        mi, ma = np.percentile(temp_diff[1]*(1-mask_pixel), [0.1, 99.9])
        ax[1].imshow(temp_diff[1], vmin=mi, vmax=ma)
        ax[1].set_title("Manual shift: %s" % shift_tmp[1])
        mi, ma = np.percentile(temp_diff[2]*(1-mask_pixel), [0.1, 99.9])
        ax[2].imshow(temp_diff[2], vmin=mi, vmax=ma)
        ax[2].set_title("Auto shift: %s" % np.round(shift_tmp[2], 2))

## Correct drift of stitching images

In [ ]:
for image_number in range(len(data.scan_id)-1):
    for key in ["images","topos"]:
        # Correct relative between image and stitching image
        temp = cci.shift_image(data[key][image_number+1].values, data["shift_"+key][image_number].values)

        data[key][image_number+1] = data[key][image_number+1].copy(data=temp)

# Execute Stitching

## Define Rois to calc scaling between holograms

In [ ]:
roi_stitch = []
fig, ax = cimshow(np.log10(im_c - np.min(im_c) + 1))
ax.set_title("Choose roi with relevant statistics for scaling")
ax.imshow(mask_pixel, alpha=0.3)

In [ ]:
# Takes start and end of x and y axis
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi_stitch.append(np.array([y1, y2, x1, x2]).astype(int))

print(f"Stitching roi:", roi_stitch)

In [ ]:
# Assign to xarray
data["roi_stitch"] = xr.DataArray(roi_stitch,dims=["stitch_pair","roi_px"])

## Calc scaling

### Define additional masks based on statistics

In [ ]:
# Plot histogram of statistics mask area
fig, ax = plt.subplots(1,2,figsize=(10, 4))
for i in range(len(data["scan_id"])):
    idx = i//2
    roi = np.s_[data["roi_stitch"].values[idx][0]:data["roi_stitch"].values[idx][1],data["roi_stitch"].values[idx][2]:data["roi_stitch"].values[idx][3]]
    
    ax[0].hist(data["images"].values[i][roi].ravel(), 200,alpha=0.2,label="Beamstop: %d"%i)
    ax[0].set_xscale("log")
    ax[0].set_yscale("log")
    ax[0].set_title("Stitch Image")
    ax[0].legend()
    ax[1].hist(data["topos"].values[i][roi].ravel(), 200,alpha=0.2,label="Beamstop: %d"%i)
    ax[1].set_title("Stitch Topo")
    ax[1].set_xscale("log")
    ax[1].set_yscale("log")
    ax[1].legend()

cimshow(im_c*(1-mask_pixel))

In [ ]:
# Find statistics pair for each combination of stitching pairs
# Pure roi mask
mask_statistics = np.zeros((len(data["scan_id"]),len(data["y"]),len(data["x"])))

for i in range(len(data["scan_id"])):
    idx = i//2
    roi = np.s_[data["roi_stitch"].values[idx][0]:data["roi_stitch"].values[idx][1],data["roi_stitch"].values[idx][2]:data["roi_stitch"].values[idx][3]]
    mask_statistics[i][roi] = 1
    mask_statistics[i] = mask_statistics[i] * (1-mask_pixel)

    mask_statistics[i] = mask_statistics[i] * (im_c[i]>10)

# to bool
mask_statistics = mask_statistics >= 1
data["mask_statistics"] = xr.DataArray(mask_statistics,dims=["scan_id","y","x"])

# Plot images with statistics mask
fig, ax = cimshow(np.log10(im_c - np.min(im_c) + 1)*(mask_statistics))

### Fitting

In [ ]:
# Fitting for images and topos
for key in ["images","topos"]:
    for i in range(len(data["scan_id"])-1):
        # Calc scaling and offset
        factor_stitch, offset_stitch = cci.dyn_factor(data[key][i].values*mask_statistics[i],
                                                    data[key][i+1].values*mask_statistics[i],
            method="correlation",
            verbose=True,
            plot=True,
        )
        
        # Stitch together
        im_stitched = data[key][i] * (1-data["mask_pixel"][i]) + factor_stitch * data[key][i+1] * data["mask_pixel"][i] * (1-data["mask_pixel"][i+1])

    data[key+"_stitched"] = xr.DataArray(im_stitched,dims=["y","x"])
    

In [ ]:
# Plot stitched image
cimshow(np.concatenate((data["images_stitched"].values[np.newaxis,...],data["images"].values),axis=0))

In [ ]:
# Plot stitched image
cimshow(np.concatenate((data["topos_stitched"].values[np.newaxis,...],data["topos"].values),axis=0))

# Image alignment between image and topo

## Set Alignment ROI 

Set a region of interest (ROI) of reference (topo) use for image registration is performed. Can include beamstop when beamstop mask was defined.

How to use:
1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
im_c = data["images_stitched"].values.copy()
topo_c = data["topos_stitched"].values.copy()

fig, ax = cimshow(im_c)
ax.set_title("Don't include the beamstop as this will misdirect the algorithm")

In [ ]:
# Takes start and end of x and y axis
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi_im_reg = np.array([y1, y2, x1, x2]).astype(int)
roi_im_reg_s = np.s_[roi_im_reg[0] : roi_im_reg[1], roi_im_reg[2] : roi_im_reg[3]]

print(f"Image registration roi:", roi_im_reg)

## Calculate drift of images

In [ ]:
shift = cci.image_registration(
    im_c[roi_im_reg_s],
    topo_c[roi_im_reg_s],
    method="dipy",
)
print("Relative shift is: %s"%shift)

In [ ]:
# Define shift manually for comparison
tmp_shift = [0.03, 0.02]

# Loop over shifts
temp_diff = np.zeros((3, im_c.shape[0], im_c.shape[1]))
shifts = [
    [0, 0],
    tmp_shift,
    -shift,
]
for i, tshift in enumerate(shifts):
    temp = cci.shift_image(im_c, tshift)
    temp_factor = cci.dyn_factor(temp, topo_c)
    temp_diff[i] = temp - temp_factor[0] * topo_c

# Plots for comparision
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12, 4))
mi, ma = np.percentile(temp_diff[0], [.1, 99.9])
ax[0].imshow(temp_diff[0], vmin=mi, vmax=ma)
ax[0].set_title("Zero shift")
mi, ma = np.percentile(temp_diff[1], [.1, 99.9])
ax[1].imshow(temp_diff[1], vmin=mi, vmax=ma)
ax[1].set_title("Manual shift: %s" % shifts[1])
mi, ma = np.percentile(temp_diff[2], [.1, 99.9])
ax[2].imshow(temp_diff[2], vmin=mi, vmax=ma)
ax[2].set_title("Auto shift: %s" % np.round(shifts[2], 2))

## Correct drift of topo

In [ ]:
# Correct relative drift
topo_c = cci.shift_image(topo_c, shift)

# Assign to xarray
data["topos_stitched"] = xr.DataArray(topo_c,dims=["y","x"])
 
# Plot original and shifted holos
mi, ma = np.percentile(np.real(im_c[im_c != 0]), (0.1, 99))
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(8, 4))
ax[0].imshow(np.real(topo), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Uncentered topo")
ax[1].imshow(np.real(topo_c), cmap="viridis", vmin=mi, vmax=ma)
ax[1].set_title("Centered topo with beamstop")

# Add circles with different radi r
tmp = np.array(image.shape) / 2
for r in np.arange(50, 200, 25):
    ax[0].add_artist(plt.Circle((tmp[1], tmp[0]), r, fill=None, ec="red"))
    ax[1].add_artist(plt.Circle((tmp[1], tmp[0]), r, fill=None, ec="red"))

# Calculate difference holograms

You can see the reconstrution of the magnetization only after subtracting the large background that you get from the diffraction on the circular object aperture (Airy Pattern). This might require a scaling factor to correct intensity changes between the hologram and the topo. Scaling factor will be determined automatically by a linear fit. If the fit seems off, there might be an issue with the data

# Create beamstops

We want to cover the beamstop with a smooth circle to cover its sharp edges as these would create ringing-like artifacts in the reconstruction plane. Make it only as large as necessary to keep as much information as possible.

## Circle beamstop

In [ ]:
# Set beamstop diameter and std for smoothing filter.
# Higher values mean stronger smoothing.
# If you have a very small beamstop you might need to reduce the smoothing value.
# Otherwise the beamstop center won't mask the direct beam with zeros
bs = interactive.InteractiveBeamstop(
    im_stitched, im_c.shape[0] / 2, im_c.shape[1] / 2, rBS=32, stdBS=3
)

In [ ]:
# Take value from widget and create beamstop mask
bs_diam = bs.rBS
bs_smoothing = bs.stdBS
mask_bs = 1 - cci.circle_mask(
    topo.shape, np.array(topo.shape) / 2, bs.rBS, sigma=bs_smoothing
)

# Apply beamstop to images
im_c = im_stitched.copy()
im_b = im_c * mask_bs

topo_c = topo_stitched.copy()
topo_b = topo_c * mask_bs

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_b, [0.1, 99.9])
ax[0].imshow(im_b, cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Masked image")

ax[1].imshow(mask_bs)
ax[1].set_title("Beamstop mask")

ax[2].imshow(1 - mask_bs)
ax[2].set_title("1 - Beamstop mask")
plt.tight_layout()

In [ ]:
# Get scaling factor and offset
factor, offset = cci.dyn_factor(
    im_b,
    topo_b,
    method="correlation",
    print_out=True,
    plot=True,
)

# Calculate differences (magnetic) and sums (topographc) contrast holograms.
# _c: centered, without beamstop, _b: centered, with beamstop
diff_c = im_c - factor * topo_c - offset
diff_b = im_b - factor * topo_b - offset
sum_c = im_c + factor * topo_c - offset
sum_b = im_b + factor * topo_b - offset

# Get scaling factor and offset
factor, offset = cci.dyn_factor(
    im_stitched,
    topo_stitched,
    method="correlation",
    print_out=True,
    plot=True,
)

# diff_stitched = im_stitched - factor * topo_stitched - offset
# sum_stitched = im_stitched + factor * topo_stitched - offset

In [ ]:
# Plot an example of the difference or sum hologram
fig, ax = cimshow(diff_b)
ax.set_title(f" Diff Id %d" % im_id)

# fig, ax = cimshow(sum_b)
# ax.set_title(f" Sum Id %d" % im_id)

# Reconstruct Diff Holos (FTH)

Reconstruct the hologramm.
1. Chose a region of interest (ROI) which means selecting one reconstruction from the rconstruction plane.
2. Propagate the image and shift the phase for maximal contrast and sharpness in your ROI
3. Optional finetuning with a widget

### Set Patterson Map ROI

Choose the reconstructions as the ROI.

1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
# Choose contrast mode
tmp = np.real(cci.reconstruct(diff_b))  # * (1 - mask_pixel)))  # magnetic contrast only
# tmp = np.real(cci.reconstruct(diff_stitched))  # magnetic contrast only
# tmp = np.real(cci.reconstruct(sum_b * (1 - mask_pixel)))  # topographic contrast only

fig, ax = cimshow(tmp, cmap="gray")

In [ ]:
# Execute to get roi
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi = np.array([y1, y2, x1, x2]).astype(int)  # ystart, ystop, xstart, xstop
# roi = [576, 871, 519, 811]
# roi = [802, 1014, 283, 488]
# roi = [613, 794, 184, 363]
roi = [354, 515, 785, 953]
# roi = [1231, 1498, 539, 844]
roi_s = np.s_[roi[0] : roi[1], roi[2] : roi[3]]
print(f"Roi Reco:{roi}")

## Tune propagation and phase
Focus the image by tuning the propagation distance. This really works like focussing in a microscope.
Phase slider will move contrast between real and imaginary part. Usually we use the phase which maximizes the contrast in the real part

In [ ]:
# Widget
slider_prop, slider_phase, button = reco.propagate(
    diff_b,  # * (1 - mask_pixel_smooth),
    roi_s,
    experimental_setup=experimental_setup,
    scale=(1, 99),
)

In [ ]:
# Read prop dist and phase from widget
prop_dist = slider_prop.value
phase = slider_phase.value

print(f"Propagation distance: %0.2f" % prop_dist)
print(f"Phase: %0.2f" % phase)

## Ugly Widget for finetuning of shift and factor
You can try to find a better scaling factor and optimize the relative shift between image and topo hologram with this widget. Just execute, find your values and then execute next cell to update the values. 

In [ ]:
# Ta-ke initial values
temp_factor = factor
temp_shift = shift.copy()
temp_phase = phase

# Open figure
fig, [ax, ax2] = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout()

# Widget
layout = ipywidgets.Layout(width="100%")
opts = dict(layout=layout)


@ipywidgets.interact(
    centery=(center[0] - 10, center[0] + 10, 0.5),
    centerx=(center[1] - 10, center[1] + 10, 0.5),
    tfactor=(
        0.6,
        1.4,
        0.0025,
    ),
    xshift=(-4, 4, 0.01),
    yshift=(-4, 4, 0.01),
    tphase=(-2 * np.pi, 2 * np.pi, 0.05),
    options=opts,
)
def update(
    centery=center[0],
    centerx=center[1],
    tfactor=factor,
    yshift=shift[0],
    xshift=shift[1],
    tphase=phase,
):  # initial values
    global temp_shift, temp_factor, temp_center, temp_diff, temp_sum, temp_phase  # ugly writing as global variable

    # Couple widget only and global variables
    temp_shift = [yshift, xshift]
    temp_factor = tfactor
    temp_center = [centery, centerx]
    temp_phase = tphase

    # Calc diff and sum holo
    shifted = cci.shift_image(im_c, temp_shift)
    temp_diff = shifted - temp_factor * topo_c
    temp_sum = shifted + temp_factor * topo_c

    temp_diff = cci.shift_image(
        temp_diff, -np.array([centery - center[0], centerx - center[1]])
    )
    temp_sum = cci.shift_image(
        temp_sum, -np.array([centery - center[0], centerx - center[1]])
    )

    temp_diff = temp_diff * mask_bs * (1 - mask_pixel_smooth)

    # Reconstruction
    recon = cci.reconstruct(
        fth.propagate(
            temp_diff,
            prop_dist * 1e-6,
            experimental_setup=experimental_setup,
        )
        * np.exp(1j * temp_phase)
    )[roi_s]

    # Plots
    vmin, vmax = np.percentile(np.real(recon), (1, 99))
    ax.imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    vmin, vmax = np.percentile(np.imag(recon), (1, 99))
    ax2.imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    # vmin, vmax = np.percentile(temp_diff * mask_bs * (1 - mask_pixel), (0.1, 99.9))
    # ax2.imshow(temp_diff * mask_bs, vmin=vmin, vmax=vmax)

In [ ]:
# Copy values from widget
factor = temp_factor
shift = temp_shift
phase = temp_phase
center = temp_center.copy()
diff_c = temp_diff.copy()
sum_c = temp_sum.copy()
diff_b = diff_c * mask_bs
sum_b = sum_c * mask_bs

## Save reconstruction

Save png files of the images and a h5 file for all important variables

In [ ]:
# Saves only real and imaginary part
holo = diff_b  # * (1 - mask_pixel_smooth)
recon = np.zeros(image.shape, dtype=np.complex_)

# Reconstruct
recon = fth.reconstruct(
    fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
    * np.exp(1j * phase)
)

# Plot
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle("Image %d - %d Stitching" % (im_id, topo_id))

vmin, vmax = np.percentile(np.real(recon[roi_s]), (0.1, 99))
t_im1 = ax[0].imshow(np.real(recon[roi_s]), vmin=vmin, vmax=vmax, cmap="gray")
ax[0].set_title("Real")
plt.colorbar(t_im1, ax=ax[0], aspect=50)

vmin, vmax = np.percentile(np.imag(recon[roi_s]), (0.1, 99))
t_im2 = ax[1].imshow(np.imag(recon[roi_s]), vmin=vmin, vmax=vmax, cmap="gray")
ax[1].set_title("Imag")
plt.colorbar(t_im2, ax=ax[1], aspect=50)

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_RefId_%04d_stitching_%s.png" % (im_id, topo_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save hdf5 file
save_fth_h5()

In [ ]:
# Closes all existing plots
plt.close("all")

# CDI Reconstruction

## Create set of pos and neg helicity holograms

In [ ]:
# Copy values from FTH reco
pos = (sum_c + diff_c) / 2
neg = (sum_c - diff_c) / 2

## Create hologram mask for phase retrieval
``mask_pixel[px,px] == 1`` means invalid, i.e., will be replaced by phase retrieval

In [ ]:
def create_auto_beamstop(im_c, topo_c, mask_draw, use_bs, bs_param):
    # Use automatically determined beamstop mask?
    if use_bs["use_bs_auto"] is True:
        mask_im = automated_beamstop(
            im_c,
            bs_param["bs_thres"],
            bs_param["bs_radius"],
            bs_param["bs_expand"],
            method=bs_param["method"],
        )
        mask_topo = automated_beamstop(
            topo_c,
            bs_param["bs_thres"],
            bs_param["bs_radius"],
            bs_param["bs_expand"],
            method=bs_param["method"],
        )
    elif use_bs["use_bs_auto"] is False:
        mask_im = mask_draw.copy()
        mask_topo = mask_draw.copy()

    # Use otsu mask as center?
    if use_bs["use_otsu"] is True:
        tmp = cci.circle_mask(
            im_c.shape, np.array(im_c.shape) / 2, auto_bs_otsu.radius, sigma=None
        )
        mask_im = (1 - tmp) * mask_im + tmp * automated_beamstop_center(im_c, 4)
        mask_topo = (1 - tmp) * mask_topo + tmp * automated_beamstop_center(
            topo_c, auto_bs_otsu.expand
        )
    elif use_bs["use_otsu"] is False:
        mask_bs_center = cci.circle_mask(
            im_c.shape,
            np.array(im_c.shape) / 2,
            bs_param["rBS"] + bs_param["stdBS"],
            sigma=None,
        )
        mask_im = mask_im + mask_bs_center
        mask_topo = mask_topo + mask_bs_center

    # Use drawn beamstop?
    if use_bs["use_draw"] is True:
        tmp = cci.circle_mask(
            im_c.shape, np.array(im_c.shape) / 2, bs_param["bs_radius"], sigma=None
        )
        mask_im = tmp * mask_im + (1 - tmp) * mask_draw
        mask_topo = tmp * mask_topo + (1 - tmp) * mask_draw

    # Saturated pixel
    mask_im = mask_im + (im_c > 65000)
    mask_topo = mask_topo + (topo_c > 65000)

    # normalize
    mask_im[mask_im > 1] = 1
    mask_topo[mask_topo > 1] = 1

    # Combine both
    mask_pixel = mask_im + mask_topo
    mask_pixel[mask_pixel > 1] = 1

    # Additional masking
    # mask_pixel[1000:1500, 450:750] = 1

    # Sharp and smooth mask
    footprint = skimage.morphology.disk(6)
    mask_pixel_smooth = skimage.morphology.dilation(mask_pixel, footprint)
    mask_pixel_smooth = gaussian_filter(mask_pixel_smooth, 2)

    return mask_im, mask_topo, mask_pixel, mask_pixel_smooth

In [ ]:
# Initial parameter for automatic beamstop detection
thres = 700
radi_thres = 140
expand = 5
method_bs = "intensity"
auto_bs = interactive.InteractiveAutoBeamstop(
    neg, thres, radi_thres, expand, method="intensity"
)

In [ ]:
# Initial parameter for automatic beamstop detection
thres = 50
radi_thres = 30
expand = 4
auto_bs_otsu = interactive.InteractiveAutoBeamstop(
    im_c, thres, radi_thres, expand, method="otsu"
)

In [ ]:
# Parameter for automatically determined beamstop
use_bs = {"use_bs_auto": False, "use_otsu": True, "use_draw": False}
bs_param = {
    "bs_thres": auto_bs.thres,
    "bs_radius": auto_bs.radius,
    "bs_expand": auto_bs.expand,
    "method": method_bs,
    "rBs": bs.rBS,
    "stdBS": bs.stdBS,
}

## Create beamstop automatically
# mask_im, mask_topo, mask_pixel, mask_pixel_smooth = create_auto_beamstop(
#    pos, neg, mask_draw, use_bs, bs_param
# )


mask_pixel = load_poly_masks(
    im_c.shape,
    ["bs_bar_ver"],
)
# footprint = skimage.morphology.disk(6)
# mask_pixel = skimage.morphology.dilation(mask_pixel, footprint)


mask_pixel_smooth = gaussian_filter(mask_pixel, 1)

mask_im = mask_pixel.copy()
mask_topo = mask_pixel.copy()

# Plot both
fig, ax = plt.subplots(2, 4, figsize=(12, 6), sharex=True, sharey=True)
mi, ma = np.percentile(pos, [1, 99.9])
ax[0, 0].imshow(pos, vmin=mi, vmax=ma)
ax[0, 0].set_title("Image")
mi, ma = np.percentile(pos * mask_im, [0.1, 99.99])
ax[0, 1].imshow(pos * mask_im, vmin=mi, vmax=ma)
ax[0, 1].set_title("Image*mask")
mi, ma = np.percentile(pos * (1 - mask_im), [0.1, 99.9])
ax[0, 2].imshow(pos * (1 - mask_im), vmin=mi, vmax=ma)
ax[0, 2].set_title("Image*(1-mask)")
ax[0, 3].imshow(mask_pixel)
ax[0, 3].set_title("Combined Mask")

mi, ma = np.percentile(neg, [1, 99.9])
ax[1, 0].imshow(neg, vmin=mi, vmax=ma)
ax[1, 0].set_title("Neg")
mi, ma = np.percentile(neg * mask_topo, [0.1, 99.99])
ax[1, 1].imshow(neg * mask_topo, vmin=mi, vmax=ma)
ax[1, 1].set_title("Neg")
mi, ma = np.percentile(neg * (1 - mask_topo), [0.1, 99.9])
ax[1, 2].imshow(neg * (1 - mask_topo), vmin=mi, vmax=ma)
ax[1, 2].set_title("Neg*(1-mask)")
mi, ma = np.percentile((pos - neg) * (1 - mask_pixel_smooth), [0.1, 99.9])
ax[1, 3].imshow((pos - neg) * (1 - mask_pixel_smooth), vmin=mi, vmax=ma)
ax[1, 3].set_title("Pos-Neg")

## Draw support mask

## Execute if you want to create a new support mask

In [ ]:
# How many references do you have?
nr_ref = 4

# Setup coordinates (nr_ref + 1 coordinates, as there is always the object aperture)
support_coordinates = [
    [pos.shape[-2] // 2, pos.shape[-1] // 2, 7] for k in range(nr_ref + 1)
]

## Execute if you want to load an existing support mask

In [ ]:
def get_supportmask_coordinates(sample):
    """
    Dictionary that stores coordinates of circular support mask apertures
    """

    # Setup dictonary
    support_coord = dict()

    # coordinates
    support_coord["s2311e-C3"] = [
        (909.5, 386.0, 89.5),
        (610.5, 156.5, 6.0),
        (1105.5, 362.0, 6.0),
        (1141.0, 398.0, 6.0),
        (1142.5, 326.5, 6.0),
        (1017.5, 745.5, 6.0),
        (1036.0, 29.5, 6.0),
        (600.0, 600.0, 6.0),
    ]

    support_coord["s2311e-C2"] = [
        (910.5, 385.5, 88.5),
        (610.5, 157.5, 7.0),
        (1105.5, 362.0, 5.5),
        (1141.0, 399.0, 6.0),
        (1142.5, 326.5, 5.5),
        (1035.9, 30.9, 5.5),
        (1017.5, 746.7, 6.5),
        (601.3, 599.6, 5.5),
    ]
    support_coord["FBIe2_cropped"] = [
        (488.5, 894.0, 5.5),
        (487.5, 714.5, 5.0),
        (603.5, 999.0, 5.0),
        (600.0, 600.0, 5.0),
        (412.0, 806.0, 67.0),
    ]
    support_coord["FBIe24"] = [
        (1344.0, 671.0, 111.0),
        (1016.8, 344.0, 9.0),
        (825.7, 539.0, 9.0),
        (827.5, 846.5, 9.0),
        (1024.5, 1024.5, 9.0),
    ]
    support_coord["FBIe11"] = [
        (1367.5, 698.5, 116.0),
        (1051.1, 367.0, 9.0),
        (1263.5, 570.9, 9.0),
        (1254.9, 802.6, 9.0),
        (1024.5, 1024.5, 9.0),
    ]

    support_coord["FBIe14"] = [
        (438.5, 865.0, 74.5),
        (627.0, 1079.0, 5.5),
        (509.5, 961.2, 5.5),
        (519.8, 765.2, 5.5),
        (648.4, 648.5, 5.5),
    ]
    return support_coord[sample]

### Create new support mask

In [ ]:
# Which supportmask to load? ("s2306a-C1", "s2308a-B1", ...)
sample = "FBIe14"

# Get coordinates and create supportmask
support_coordinates = get_supportmask_coordinates(sample)

In [ ]:
ds = interactive.InteractiveCircleCoordinates(
    np.real(
        fth.reconstruct(pos * mask_bs * np.exp(1j * phase) * (1 - mask_pixel_smooth))
    ),
    len(support_coordinates),
    coordinates=support_coordinates,
)

In [ ]:
# Take coordinates of circles from widget
support_coordinates = ds.get_mask()

# Create supportmask
supportmask = create_supportmask(support_coordinates, pos.shape)

# What to plot?
tmp = np.real(
    fth.reconstruct(pos * mask_bs * np.exp(1j * phase) * (1 - mask_pixel_smooth))
)

# Plot supportmask as overlay
fig, ax = plt.subplots(figsize=(6, 6))
mi, ma = np.percentile(tmp, (1, 99))
ax.imshow(tmp, vmin=mi, vmax=ma, cmap="gray")
ax.imshow(supportmask, alpha=0.3, cmap="binary")
ax.set_title("Image with overlayed mask")

### Take Roi

In [ ]:
roi_cdi = interactive.axis_to_roi(ax)
# roi_cdi = [807, 1006, 285, 481]
# roi_cdi = [802, 1012, 279, 489]
# roi_cdi = [334, 488, 725, 884]
# roi_cdi = [1204, 1474, 539, 809]
roi_cdi = [360, 517, 785, 946]
roi_cdi = np.s_[roi_cdi[0] : roi_cdi[1], roi_cdi[2] : roi_cdi[3]]
print("Sliced roi:", roi_cdi)

## Do Phase Retrieval

In [ ]:
(
    retrieved_p,
    retrieved_n,
    retrieved_p_pc,
    retrieved_n_pc,
    bsmask_p,
    bsmask_n,
    gamma_p,
    gamma_n,
) = phase_retrieval(pos, neg, mask_pixel, supportmask, Startimage=None, Startgamma=None)

## Reconstruct images

In [ ]:
# New beamstop for CDI recos as phase retrievel of low-q might be poor
use_bs_reco = False
bs_diam_cdi = 45  # 30#50

# Create beamstop
if use_bs_reco is True:
    mask_bs_cdi = 1 - cci.circle_mask(
        topo.shape, np.array(topo.shape) / 2, bs_diam_cdi, sigma=4
    )
    mask_bs_cdi = 1 - mask_pixel_smooth.copy()
elif use_bs_reco is False:
    mask_bs_cdi = np.ones(pos.shape)  # if you don't want a beamstop

# Get Recos
# Positive helicity
p = fth.reconstructCDI(
    fth.propagate(
        retrieved_p * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# Negative helicity
n = fth.reconstructCDI(
    fth.propagate(
        retrieved_n * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)
# p = fth.sub_pixel_centering(p, dx, dy)
# n = fth.sub_pixel_centering(n, dx, dy)


# Get Recos partial coherence
# Positiv partial coherence
p_pc = fth.reconstructCDI(
    fth.propagate(
        retrieved_p_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# Negative partial coherence
n_pc = fth.reconstructCDI(
    fth.propagate(
        retrieved_n_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)
# p_pc = fth.sub_pixel_centering(p_pc, dx, dy)
# n_pc = fth.sub_pixel_centering(n_pc, dx, dy)

# optimize phase
# recon = p - n
recon = p_pc - n_pc  # / (p_pc + n_pc)
_, phase_cdi = optimize_phase_contrast(
    recon, supportmask, method="contrast", prefered_color="white"
)

# Plotting
mode = "-"
slider_prop, slider_phase, slider_dx, slider_dy = rec.focusCDI(
    retrieved_p_pc * mask_bs_cdi,
    retrieved_n_pc * mask_bs_cdi,
    roi_cdi,
    mask=supportmask,
    phase=phase_cdi,
    dx=dx,
    dy=dy,
    prop_dist=prop_dist_cdi,
    experimental_setup=experimental_setup,
    operation=mode,
    max_prop_dist=10,
    scale=(0.1, 99.9),
)

In [ ]:
# Get phase from slider
phase_cdi = slider_phase.value
prop_dist_cdi = slider_prop.value

# Apply phase
p = p * np.exp(1j * phase_cdi)
n = p * np.exp(1j * phase_cdi)
p_pc = p_pc * np.exp(1j * phase_cdi)
n_pc = n_pc * np.exp(1j * phase_cdi)

print("Phase CDI: %s" % phase_cdi)
print("Prop_dist: %s" % prop_dist_cdi)

In [ ]:
# Confirm that offset subtraction works, i.e., only small fraction of hologram is actually masked
fig, ax = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
tmp = np.abs(retrieved_p_pc * mask_bs_cdi)
mi, ma = np.percentile(tmp, [0.1, 99.9])
ax[0, 0].imshow(tmp, vmin=mi, vmax=ma)
ax[0, 0].set_title("Pos holo")

tmp = np.abs(retrieved_n_pc)
mi, ma = np.percentile(tmp, [0.1, 99.9])
ax[0, 1].imshow(tmp, vmin=mi, vmax=ma)
ax[0, 1].set_title("Neg holo")
ax[1, 0].imshow(bsmask_p)
ax[1, 0].set_title("Pos holo mask")
ax[1, 1].imshow(bsmask_n)
ax[1, 1].set_title("Neg holo mask")

## Save

In [ ]:
# Magnetic field value
data_key = "magnett_read"
values = np.mean(np.array(load_data(im_id, data_key)) * 1000)
values = [np.round(values, 2)]

# Cryostat temperature if saved in nxs file
try:
    data_key = "cryob"
    values.append(np.mean(np.array(load_collection(im_id, data_key))))
    title = "Image %d - %d CDI @%.2f mT, %0.1f K" % (
        im_id,
        topo_id,
        values[0],
        values[1],
    )
except:
    title = "Image %d - %d CDI @%.2f mT" % (im_id, topo_id, values[0])

In [ ]:
# Saves only real and imaginary part
recon_all = p - n
recon_all = p_pc - n_pc  # / (p_pc + n_pc)
recon = recon_all.copy()

# Plot
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle(title)

vmin, vmax = np.percentile(np.real(recon[roi_cdi]), (0.1, 99.9))
t_im1 = ax[0].imshow(np.real(recon[roi_cdi]), vmin=vmin, vmax=vmax, cmap="gray")
ax[0].set_title("Real")
plt.colorbar(t_im1, ax=ax[0], aspect=50)

vmin, vmax = np.percentile(np.imag(recon[roi_cdi]), (0.1, 99.9))
t_im2 = ax[1].imshow(np.imag(recon[roi_cdi]), vmin=vmin, vmax=vmax, cmap="gray")
ax[1].set_title("Imag")
plt.colorbar(t_im2, ax=ax[1], aspect=50)

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_RefId_%s_cdi_diff_stitching_%s.png" % (im_id, topo_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save h5
save_cdi_h5()

# Batch processing CDI

## Define Scan Ids

In [ ]:
# Load support mask of which sample?
sample = "FBIe14"

In [ ]:
# Define the sets for reconstructions. You can make a list or use np.arange
# im_id_set should always have ids of positive helicity holograms,
# topo_id_set those of negative helicity or a proper topo

im_id_set = np.arange(3365, 3367 + 1)
im_id_set = [3373, 3374]
topo_id_set = 3370 * np.ones(len(im_id_set), dtype=int)

# In case of single helicity reconstructions, adapt the helicity
# for contrast inversion
helicity = 1 * np.ones(len(im_id_set), dtype=int)  # [1,-1]

# Do cdi?
do_cdi = True

print("Dynamics Set:  %s" % im_id_set)
print("Reference Set: %s" % topo_id_set)
print("Helicity: %s" % helicity)

## Execute Stack Reconstruction

In [ ]:
# Ugly Automatic processing of image stacks
recons_name = []  # for gifs
for it, im_id in enumerate(im_id_set):
    # Load images
    image, _ = load_processing(im_id)

    # Get topo
    # Do you want to construct topo holo from two helicity images
    try:
        len(topo_id_set[it])
    except:
        # Usual case
        # Get also topo & dark id from list of sets
        topo_id = topo_id_set[it]

        # Load data
        print(f"Loading imageId: %04d, topoId: %04d" % (im_id, topo_id))
        topo, _ = load_processing(topo_id)

        # Process images
        worker_dict = worker(image, topo)

        # Save topo hologram
        save_topo_holo(worker_dict["sum_c"], im_id, topo_id)
    else:
        print("Using Topo from sum of two helicity holograms")
        pos_id = topo_id_set[it][0]
        neg_id = topo_id_set[it][1]
        topo_id = topo_id_set[it]

        try:
            topo = load_topo_holo(pos_id, neg_id) / 2
        except:
            topo = load_topo_holo(neg_id, pos_id) / 2
        topo = cci.shift_image(
            topo, -shift_c
        )  # shift out of center so you don't need to change the worker

        # Process images
        worker_dict = worker(image, topo)

    # Save FTH reco
    # Magnetic field value
    data_key = "magnett_read"
    values = np.mean(np.array(load_data(im_id, data_key)) * 1000)
    values = [np.round(values, 2)]

    # Cryostat temperature if saved in nxs file
    try:
        data_key = "cryob"
        values.append(np.mean(np.array(load_collection(im_id, data_key))))
        title = "Image %d - %s @%.2f mT, %0.1f K" % (
            im_id,
            topo_id,
            values[0],
            values[1],
        )
    except:
        title = "Image %d - %s @%.2f mT" % (im_id, topo_id, values[0])

    # Reconstruct
    recon = fth.reconstruct(
        fth.propagate(
            worker_dict["holo"], prop_dist * 1e-6, experimental_setup=experimental_setup
        )
        * np.exp(1j * phase)
    )

    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(title)

    vmin, vmax = np.percentile(
        np.real(recon[roi[0] : roi[1], roi[2] : roi[3]]), (1, 99)
    )
    t_im1 = ax[0].imshow(
        np.real(recon[roi[0] : roi[1], roi[2] : roi[3]]),
        vmin=vmin,
        vmax=vmax,
        cmap="gray",
    )
    ax[0].set_title("Real")
    plt.colorbar(t_im1, ax=ax[0], aspect=50)

    vmin, vmax = np.percentile(
        np.imag(recon[roi[0] : roi[1], roi[2] : roi[3]]), (1, 99)
    )
    t_im2 = ax[1].imshow(
        np.imag(recon[roi[0] : roi[1], roi[2] : roi[3]]),
        vmin=vmin,
        vmax=vmax,
        cmap="gray",
    )
    ax[1].set_title("Imag")
    plt.colorbar(t_im2, ax=ax[1], aspect=50)

    # Save images
    fname = join(
        folder_general,
        "Recon_ImId_%04d_RefId_%s_fth_diff_stack_%s.png" % (im_id, topo_id, USER),
    )
    print("Saving: %s" % fname)
    plt.savefig(fname, bbox_inches="tight", transparent=False)

    ################ CDI ###############
    if do_cdi is True:
        # Create pos and neg helicity set
        pos = (worker_dict["sum_c"] + worker_dict["diff_c"]) / 2
        neg = (worker_dict["sum_c"] - worker_dict["diff_c"]) / 2

        # Create beamstop automatically
        mask_im, mask_topo, mask_pixel, mask_pixel_smooth = create_auto_beamstop(
            pos, neg, mask_draw, use_bs, bs_param
        )

        fig, ax = cimshow(mask_pixel.astype(int))
        ax.set_title("Verify that this looks like an acceptable beamstop")

        # Get coordinates and create supportmask
        support_coordinates = get_supportmask_coordinates(sample)
        supportmask = create_supportmask(support_coordinates, pos.shape)

        # Do phase retrieval
        (
            retrieved_p,
            retrieved_n,
            retrieved_p_pc,
            retrieved_n_pc,
            bsmask_p,
            bsmask_n,
            gamma_p,
            gamma_n,
        ) = phase_retrieval(
            pos, neg, mask_pixel, supportmask, Startimage=None, Startgamma=None
        )

        # Get Recos partial coherence
        # Positiv partial coherence
        p_pc = fth.reconstructCDI(
            fth.propagate(
                retrieved_p_pc * mask_bs_cdi,
                prop_dist_cdi * 1e-6,
                experimental_setup=experimental_setup,
            )
        )
        # Negative partial coherence
        n_pc = fth.reconstructCDI(
            fth.propagate(
                retrieved_n_pc * mask_bs_cdi,
                prop_dist_cdi * 1e-6,
                experimental_setup=experimental_setup,
            )
        )

        ##### Calc reco and optimze contrast
        recon = helicity[it] * (p_pc - n_pc)
        _, phase_cdi = optimize_phase_contrast(
            recon,
            supportmask,
            method="contrast",
            prefered_color="white",
        )
        # phase_cdi = 0
        recon = recon * np.exp(1j * phase_cdi)
        print("Phase is:", np.round(phase_cdi, 2))
        ########

        # Plot
        fig, ax = plt.subplots(1, 2, figsize=(10, 4))
        fig.suptitle(title)

        vmin, vmax = np.percentile(np.real(recon[roi_cdi]), (0.5, 99.5))
        t_im1 = ax[0].imshow(
            np.real(recon[roi_cdi]),
            vmin=vmin,
            vmax=vmax,
            cmap="gray",
        )
        ax[0].set_title("Real")
        plt.colorbar(t_im1, ax=ax[0], aspect=50)

        vmin, vmax = np.percentile(np.imag(recon[roi_cdi]), (0.5, 99.5))
        t_im2 = ax[1].imshow(np.imag(recon[roi_cdi]), vmin=vmin, vmax=vmax, cmap="gray")
        ax[1].set_title("Imag")
        plt.colorbar(t_im2, ax=ax[1], aspect=50)

        # Save images
        fname = join(
            folder_general,
            "Recon_ImId_%04d_RefId_%s_cdi_stack_%s.png" % (im_id, topo_id, USER),
        )

        print("Saving: %s" % fname)
        plt.savefig(fname, bbox_inches="tight", transparent=False)
        recons_name.append(fname)

        # Save files as h5
        save_cdi_h5()
    else:
        print("Phase retrieval disabled!")

    print(" ")
print("CDI stack processing finished")

In [ ]:
plt.close("all")

# Gifs

In [ ]:
im_id_set = [
    1268,
    1269,
    1272,
    1273,
    1276,
    1277,
    1280,
    1281,
    1284,
    1285,
    1288,
    1289,
    1292,
    1293,
    1296,
    1297,
    1300,
    1301,
    1303,
    1306,
    1308,
    1313,
    1314,
    1315,
    1316,
    1317,
]
topo_id_set = [
    1267,
    1270,
    1271,
    1274,
    1275,
    1278,
    1279,
    1282,
    1283,
    1286,
    1287,
    1290,
    1291,
    1294,
    1295,
    1298,
    1299,
    1302,
    [1299, 1300],
    1307,
    [1306, 1307],
    1312,
    [1312, 1313],
    [1312, 1313],
    [1312, 1313],
    [1312, 1313],
]

recons_name = []
for i, im_id in enumerate(im_id_set):
    fname = join(
        folder_general,
        "Recon_ImId_%04d_RefId_%s_%s_cdi_diff_stack.png"
        % (im_id, topo_id_set[i], USER),
    )
    recons_name.append(fname)

# Create gif of last scan
var = [imageio.imread(file) for file in recons_name]
fname = f"ImId_%04d_%04d_%s.gif" % (im_id_set[0], im_id_set[-1], USER)
gif_path = path.join(folder_general, fname)
print("Saving gif:%s" % gif_path)
imageio.mimsave(gif_path, var, fps=2)
print("Done!")